In [170]:
import re
import pandas as pd
import numpy as np
from scipy.sparse.linalg import norm

In [257]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop

    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

###### 【問題1】BoWとN-gram(手計算)
特殊文字除去(!や〜など)、単語分割をし以下の2パターンで文1〜文4を数値化(ベクトル化)してください。
BoW(1-gram)
BoW(2-gram)
手計算の後見やすい形にしてください。

In [2]:
vocabulary = ["I", "love", "this", "is", "the", "baseball"]
ms_kk_texts = ["I love baseball !!", "I love this !"]
texts_vec = [[1,1,0,0,0,1], [1,1,1,0,0,0]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram

,I,love,this,is,the,baseball
I love baseball !!,1,1,0,0,0,1
I love this !,1,1,1,0,0,0


文1: 今撮影中で〜す！
文2: 今休憩中で〜す(^^)
文3: 今日ドラマ撮影で〜す！
文4: 今日、映画瞬公開で〜す！！！

###### ベクトル化

BoW(1-gram)

In [3]:
vocabulary = ["今", "今日", "撮影", "映画", "ドラマ", "瞬","休憩", "公開", "中", "です"]
ms_kk_texts = ["今撮影中で〜す！", "今休憩中で〜す(^^)", "今日ドラマ撮影で〜す！", "今日、映画瞬公開で〜す！！！"]
texts_vec = [[1,0,1,0,0,0,0,0,1,1], [1,0,0,0,0,0,1,0,1,1], [0,1,1,0,1,0,0,0,0,1], [0,1,0,1,0,1,0,1,0,1]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram

,今,今日,撮影,映画,ドラマ,瞬,休憩,公開,中,です
今撮影中で〜す！,1,0,1,0,0,0,0,0,1,1
今休憩中で〜す(^^),1,0,0,0,0,0,1,0,1,1
今日ドラマ撮影で〜す！,0,1,1,0,1,0,0,0,0,1
今日、映画瞬公開で〜す！！！,0,1,0,1,0,1,0,1,0,1


BoW(2-gram)

In [71]:
vocabulary = ["今", "今日", "撮影", "映画", "ドラマ", "映画", "休憩", "公開", "中", "です"]
ms_kk_texts = ["今撮影中で〜す！", "今休憩中で〜す(^^)", "今日ドラマ撮影で〜す！", "今日、映画瞬公開で〜す！！！"]

words = ["今","撮影","中","です","今","休憩","中","です", "今日","ドラマ","撮影","です", "今日","映画","瞬","公開","です"]

n=2
N_gram_2_list = list(zip(*(words[i:] for i in range(n))))
N_gram_2_list = list(set(N_gram_2_list))

df_bow_2gram = pd.DataFrame(data = None, columns = N_gram_2_list, index = ms_kk_texts)
df_bow_2gram[0:1]

,"(今, 休憩)","(ドラマ, 撮影)","(中, です)","(撮影, 中)","(映画, 瞬)","(瞬, 公開)","(公開, です)","(今日, ドラマ)","(今日, 映画)","(休憩, 中)","(今, 撮影)","(撮影, です)","(です, 今)","(です, 今日)"
今撮影中で〜す！,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
texts_vec = [[0,0,0,0,0,0,0,1,0,0,0,1,0,1], 
             [1,0,0,0,0,0,1,0,0,0,0,0,0,1], 
             [0,0,0,1,0,0,0,0,1,0,1,0,0,0], 
             [0,0,1,0,1,1,0,0,0,1,0,0,0,0]]

In [6]:
df_bow_2gram = pd.DataFrame(data = texts_vec, columns = N_gram_2_list, index = ms_kk_texts)
df_bow_2gram

,"(今, 休憩)","(ドラマ, 撮影)","(中, です)","(撮影, 中)","(映画, 瞬)","(瞬, 公開)","(公開, です)","(今日, ドラマ)","(今日, 映画)","(休憩, 中)","(今, 撮影)","(撮影, です)","(です, 今)","(です, 今日)"
今撮影中で〜す！,0,0,0,0,0,0,0,1,0,0,0,1,0,1
今休憩中で〜す(^^),1,0,0,0,0,0,1,0,0,0,0,0,0,1
今日ドラマ撮影で〜す！,0,0,0,1,0,0,0,0,1,0,1,0,0,0
今日、映画瞬公開で〜す！！！,0,0,1,0,1,1,0,0,0,1,0,0,0,0


###### 【問題2】TF-IDF(手計算)
問題1のコーパスを使って、文1〜文4をTFIDFで数値化(ベクトル化)してください。 
問題1と同様、手計算の後見やすい形にしてください。

In [77]:
sum_df = df_bow_1gram.sum(axis=0)
sum_nsd = df_bow_1gram.sum(axis=1)
N = len(df_bow_1gram.index)

In [78]:
sum_df

今      2
今日     2
撮影     2
映画     1
ドラマ    1
瞬      1
休憩     1
公開     1
中      2
です     4
dtype: int64

In [79]:
sum_nsd

今撮影中で〜す！          4
今休憩中で〜す(^^)       4
今日ドラマ撮影で〜す！       4
今日、映画瞬公開で〜す！！！    5
dtype: int64

In [80]:
tf = df_bow_1gram.div(sum_nsd.values, axis=0)

In [81]:
idf = np.log2( N / sum_df )
#idf[idf == np.inf] = 0

In [82]:
tf

,今,今日,撮影,映画,ドラマ,瞬,休憩,公開,中,です
今撮影中で〜す！,0.25,0.00,0.25,0.0,0.00,0.0,0.00,0.0,0.25,0.25
今休憩中で〜す(^^),0.25,0.00,0.00,0.0,0.00,0.0,0.25,0.0,0.25,0.25
今日ドラマ撮影で〜す！,0.00,0.25,0.25,0.0,0.25,0.0,0.00,0.0,0.00,0.25
今日、映画瞬公開で〜す！！！,0.00,0.20,0.00,0.2,0.00,0.2,0.00,0.2,0.00,0.20


In [83]:
idf

今      1.0
今日     1.0
撮影     1.0
映画     2.0
ドラマ    2.0
瞬      2.0
休憩     2.0
公開     2.0
中      1.0
です     0.0
dtype: float64

In [84]:
TFIDF = tf * idf
TFIDF

,今,今日,撮影,映画,ドラマ,瞬,休憩,公開,中,です
今撮影中で〜す！,0.25,0.00,0.25,0.0,0.0,0.0,0.0,0.0,0.25,0.0
今休憩中で〜す(^^),0.25,0.00,0.00,0.0,0.0,0.0,0.5,0.0,0.25,0.0
今日ドラマ撮影で〜す！,0.00,0.25,0.25,0.0,0.5,0.0,0.0,0.0,0.00,0.0
今日、映画瞬公開で〜す！！！,0.00,0.20,0.00,0.4,0.0,0.4,0.0,0.4,0.00,0.0


###### 【問題3】テキストクリーニング(プログラミング)
このテキストに以下の処理を施してください。
urlを削除  
【〇〇】を削除  
改行等の特殊文字を削除  
絵文字除去  
ここではしませんが、数字を文字列NUMBERに置き換える処理をよくします。 

In [14]:
text_noro = " <!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:"

In [15]:
def remove_URL(text):
    cleaned = re.sub(r"(https?|ftp)(:\/\/[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+\$,%#]+)", "" ,str(text))
    
    return cleaned

In [16]:
def remove_bracket(text):
    cleaned = re.sub("(?<=【).*?(?=】)", "" ,str(text))

    return cleaned 

In [35]:
def remove_bracket2(text):
    cleaned = re.sub("(?<=\<).*?(?=\>)", "" ,str(text))

    return cleaned 

In [ ]:
def remove_bracket3(text):
    cleaned = re.sub("(?<=\{).*?(?=\})", "" ,str(text))

    return cleaned 

In [17]:
def remove_emoji(text):
    cleaned = re.sub("(?<=:).*?(?=:)", "" ,str(text))

    return cleaned 

In [60]:
def normalize_number(text):
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

In [367]:
def normal_char_pickup(text):
    cleaned = re.findall('[ー0-9a-zA-Zあ-んア-ン一-鿐\s]', text)
    
    return cleaned

In [61]:
def text_cleaning(text):
    tmp = remove_URL(text)
    tmp = remove_bracket(tmp)
    tmp = remove_bracket2(tmp)
    tmp = remove_emoji(tmp)
    tmp = normalize_number(tmp)
    tmp = normal_char_pickup(tmp)
    tmp = ','.join(tmp)
    cleaned = tmp.replace(",", "")
    return cleaned

In [36]:
tmp = remove_URL(text_noro)

In [37]:
tmp = remove_bracket(tmp)

In [38]:
tmp = remove_bracket2(tmp)

In [39]:
tmp = remove_emoji(tmp)
tmp

' <> *【】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<>\n\nPyCharm\n<>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ ::'

In [40]:
ret = normal_char_pickup(tmp)
text_cleaned = ','.join(ret)
text_cleaned = text_cleaned.replace(",", "")

In [41]:
text_cleaned

'有償のRubyMineやPyCharmの6月間100OFFクーポンをご希望者の方先着100名様に贈呈いたしますこの度RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り100クーポンをいただくことができましたRubyMinePyCharmご希望の方は手を挙げて方式でご希望の方はこの投稿の手あげスタンプをクリックしてください期限は2019年3月20日水2200までとさせていただきますふるってのご希望をお待ちしております'

###### 【問題4】形態素解析
上記のクリーニングしたテキストをJanomeを用いて形態素解析をし、
名詞または動詞の単語を抜き出してください。

In [42]:
from janome.tokenizer import Tokenizer

In [43]:
words = []
t = Tokenizer()
for token in t.tokenize(text_cleaned):
    if token.part_of_speech.split(',')[0] == "名詞" or token.part_of_speech.split(',')[0] == "動詞":
        words.append(token.surface)
        print(token)

有償	名詞,一般,*,*,*,*,有償,ユウショウ,ユーショー
RubyMine	名詞,一般,*,*,*,*,RubyMine,*,*
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
6	名詞,数,*,*,*,*,6,*,*
月間	名詞,一般,*,*,*,*,月間,ゲッカン,ゲッカン
100	名詞,数,*,*,*,*,100,*,*
OFF	名詞,一般,*,*,*,*,OFF,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
希望	名詞,サ変接続,*,*,*,*,希望,キボウ,キボー
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
方	名詞,非自立,一般,*,*,*,方,ホウ,ホー
先着	名詞,サ変接続,*,*,*,*,先着,センチャク,センチャク
100	名詞,数,*,*,*,*,100,*,*
名	名詞,接尾,助数詞,*,*,*,名,メイ,メイ
様	名詞,接尾,人名,*,*,*,様,サマ,サマ
贈呈	名詞,サ変接続,*,*,*,*,贈呈,ゾウテイ,ゾーテイ
いたし	動詞,非自立,*,*,五段・サ行,連用形,いたす,イタシ,イタシ
度	名詞,非自立,副詞可能,*,*,*,度,タビ,タビ
RubyMine	名詞,一般,*,*,*,*,RubyMine,*,*
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
メーカー	名詞,一般,*,*,*,*,メーカー,メーカー,メーカー
JetBrains	名詞,一般,*,*,*,*,JetBrains,*,*
社	名詞,接尾,一般,*,*,*,社,シャ,シャ
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
コード	名詞,一般,*,*,*,*,コード,コード,コード
提供	名詞,サ変接続,*,*,*,*,提供,テイキョウ,テイキョー
交渉	名詞,サ変接続,*,*,*,*,交渉,コウショウ,コーショー
実り	動詞,自立,*,*,五段・ラ行,連用形,実る,ミノリ,ミノリ
100	名詞,数,*,*,*,*,100,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
いただく	動詞,自立,*,*,五段・カ行イ音便,基本形,いただく,イタダク,イタダク
こと	名詞,非自立,一般,*,*,*,こと,コト,コト
でき	動詞,自立,*,

In [26]:
words

['everyone',
 '有償',
 'RubyMine',
 'PyCharm',
 '6',
 '月間',
 '100',
 'OFF',
 'クーポン',
 '希望',
 '者',
 '方',
 '先着',
 '100',
 '名',
 '様',
 '贈呈',
 'いたし',
 '度',
 'RubyMine',
 'PyCharm',
 'メーカー',
 'JetBrains',
 '社',
 'クーポン',
 'コード',
 '提供',
 '交渉',
 '実り',
 '100',
 'クーポン',
 'いただく',
 'こと',
 'でき',
 'RubyMinePyCharm',
 '希望',
 '方',
 '手',
 '挙げ',
 '方式',
 '希望',
 '方',
 '投稿',
 '手',
 'あげ',
 'スタンプ',
 'クリック',
 'し',
 'ください',
 '期限',
 '2019',
 '年',
 '3',
 '月',
 '20',
 '日',
 '水',
 '2200',
 'さ',
 'せ',
 'いただき',
 '希望',
 'お待ち',
 'し',
 'おり']

###### 【問題5】ニュースの分析
以下の流れでニュースを分析してください。
まずどんなニュースなのか読んでみる   

In [27]:
from sklearn.datasets import load_files
# encodingをutf-8指定して読み込み
bin_data = load_files('./text', encoding='utf-8')
documents = bin_data.data

In [28]:
len(documents)

7376

In [29]:
documents[1010]

'http://news.livedoor.com/article/detail/5695257/\n2011-08-24T11:00:00+0900\n「あなたのモットーは？」に即答できますか？-人事担当者がこっそり教える採用ウラ話 vol.7\n転職者なら誰でも気になる採用する側の心理。しかし、実際に人事担当者から本音を聞きだすことはなかなかできないもの。そこでこのコーナーでは、多くの応募者を見てきた人事担当者に、意外な“採用ポイント”や心に残る採用エピソードなどをこっそり教えていただいた。\n\n\n「あなたのモットーは？」に即答できますか？\n\n\nアウトソーシング業界\n人事担当\n山下 雄三氏（仮名）\n私たち人事の仕事は、一人ひとりの人材の"目に見えない部分"を評価することだと思っています。資格や実務経験の有無といった目に見える部分は誰だって書類を見ればわかる。人事は採用のプロとして、それ以外のメンタル的な部分や潜在能力を見極め、適材適所を心掛けなければならないんです。ですから私は職務経歴書や履歴書はあまり重視しません。必要最低限の条件さえ満たしていれば面接で直接お会いして判断するようにしています。\n\nしかし面接でのジャッジは早いですよ。たいていの場合、数分でその人の"素養"はわかります。たとえば、差し出した名刺を着座したまま受け取ったり、会話中こちらの目を見なかったり、そうした些細なところにもその人の人柄というものは表れてくるんです。\n\nそういえば以前、ボロボロのジーンズに雪駄ばきという格好で面接を受けに来た方がいました。エンジニアだと私服で面接に来る方は多いんですが、これにはびっくりしましたよ。これ見よがしに個性を主張しようというならまだ分かるのですが（それも好ましくはありませんが）、どうやら本気でその格好がいいと思っているようで…。 でもやっぱり清潔感がないのはまずいですよねぇ（笑）。\n\n他にも「面接を受けるときは清潔感のあるスーツで」「面接官に進められるまで着席しない」といった新卒のときは当たり前だったマナーをすっかり忘れてしまっている人は結構います。でもこれは企業の責任でもあります。本当に魅力的な企業で、「どうしてもこの会社に入りたい」と思われるようならこうした態度はとられないでしょう？ マナーはその人の熱意の表れでもありますか

###### 出現単語をカウントして分析する  

In [45]:
#Documnet = 1010
words = []
t = Tokenizer()
for token in t.tokenize(documents[1010]):
    if token.part_of_speech.split(',')[0] == "名詞" or token.part_of_speech.split(',')[0] == "動詞":
        words.append(token.surface)
        print(token)
        
        
print("\n単語数:",len(words))

http	名詞,固有名詞,組織,*,*,*,http,*,*
://	名詞,サ変接続,*,*,*,*,://,*,*
news	名詞,一般,*,*,*,*,news,*,*
.	名詞,サ変接続,*,*,*,*,.,*,*
livedoor	名詞,一般,*,*,*,*,livedoor,*,*
.	名詞,サ変接続,*,*,*,*,.,*,*
com	名詞,一般,*,*,*,*,com,*,*
/	名詞,サ変接続,*,*,*,*,/,*,*
article	名詞,一般,*,*,*,*,article,*,*
/	名詞,サ変接続,*,*,*,*,/,*,*
detail	名詞,一般,*,*,*,*,detail,*,*
/	名詞,サ変接続,*,*,*,*,/,*,*
5695257	名詞,数,*,*,*,*,5695257,*,*
/	名詞,サ変接続,*,*,*,*,/,*,*
2011	名詞,数,*,*,*,*,2011,*,*
-	名詞,サ変接続,*,*,*,*,-,*,*
08	名詞,数,*,*,*,*,08,*,*
-	名詞,サ変接続,*,*,*,*,-,*,*
24	名詞,数,*,*,*,*,24,*,*
T	名詞,一般,*,*,*,*,T,*,*
11	名詞,数,*,*,*,*,11,*,*
:	名詞,サ変接続,*,*,*,*,:,*,*
00	名詞,数,*,*,*,*,00,*,*
:	名詞,サ変接続,*,*,*,*,:,*,*
00	名詞,数,*,*,*,*,00,*,*
+	名詞,サ変接続,*,*,*,*,+,*,*
0900	名詞,数,*,*,*,*,0900,*,*
あなた	名詞,代名詞,一般,*,*,*,あなた,アナタ,アナタ
モットー	名詞,一般,*,*,*,*,モットー,モットー,モットー
即答	名詞,サ変接続,*,*,*,*,即答,ソクトウ,ソクトー
でき	動詞,自立,*,*,一段,連用形,できる,デキ,デキ
？-	名詞,サ変接続,*,*,*,*,？-,*,*
人事	名詞,一般,*,*,*,*,人事,ジンジ,ジンジ
担当	名詞,サ変接続,*,*,*,*,担当,タントウ,タントー
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
教える	動詞,自立,*,*,一段,基本形,教える,オシエル,オシエル
採用	名詞,サ変接続,*,*,*,*,採用,サ

###### テキストをクリーニングする  

In [119]:
doc  = [0 for i in range(len(documents))]
for i in range(len(documents)):
    doc[i] = text_cleaning(documents[i])

In [120]:
doc[1010]

'000T000あなたのモットーはに即答できますか人事担当者がこっそり教える採用ウラ話vol0転職者なら誰でも気になる採用する側の心理しかし実際に人事担当者から本音を聞きだすことはなかなかできないものそこでこのコーナーでは多くの応募者を見てきた人事担当者に意外な採用ポイントや心に残る採用エピソードなどをこっそり教えていただいたあなたのモットーはに即答できますかアウトソーシング業界人事担当山下雄三氏仮名私たち人事の仕事は一人ひとりの人材の目に見えない部分を評価することだと思っています資格や実務経験の有無といった目に見える部分は誰だって書類を見ればわかる人事は採用のプロとしてそれ以外のメンタル的な部分や潜在能力を見極め適材適所を心掛けなければならないんですですから私は職務経歴書や履歴書はあまり重視しません必要最低限の条件さえ満たしていれば面接で直接お会いして判断するようにしていますしかし面接でのジャッジは早いですよたいていの場合数分でその人の素養はわかりますたとえば差し出した名刺を着座したまま受け取ったり会話中こちらの目を見なかったりそうした些細なところにもその人の人柄というものは表れてくるんですそういえば以前ボロボロのジーンズに雪駄ばきという格好で面接を受けに来た方がいましたエンジニアだと私服で面接に来る方は多いんですがこれにはびっくりしましたよこれ見よがしに個性を主張しようというならまだ分かるのですがそれも好ましくはありませんがどうやら本気でその格好がいいと思っているようででもやっぱり清潔感がないのはまずいですよねぇ笑他にも面接を受けるときは清潔感のあるスーツで面接官に進められるまで着席しないといった新卒のときは当たり前だったマナーをすっかり忘れてしまっている人は結構いますでもこれは企業の責任でもあります本当に魅力的な企業でどうしてもこの会社に入りたいと思われるようならこうした態度はとられないでしょうマナーはその人の熱意の表れでもありますからね悪い例ばかり挙げてしまいましたが好印象を与える人の特徴としてはこちらの質問に対してきちっと適切に回答してくれるということがあると思いますなんだそんなことと思うかもしれませんがこれは結構大切なことですこちらの質問の意図をしっかり理解して難しい技術の話でもわかりやすく伝えるというのはなかなか難しいことでそれができる人はそう多くはな

###### BoW + TFIDFでベクトル化する  

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [92]:
def analyzer(text):
    words_list = []
    t = Tokenizer()
    for token in t.tokenize(text):
        if token.part_of_speech.split(',')[0] == "名詞" or token.part_of_speech.split(',')[0] == "動詞":
            words_list.append(token.surface)
            
    return words_list

In [113]:
def text_vectorizer(text):
    vectorizer = TfidfVectorizer(analyzer=analyzer)
    ret = vectorizer.fit_transform(text)
    return vectorizer

In [121]:
vector = text_vectorizer(doc)

['0', '00', '000', '0000', '00000', '000000', '0000000', '00000000', '0000000000', '0000000000000', '00000000000000', '0000000000000000', '00000000000000000', '00000000000000000000', '000000000000000000000', 'A', 'AA', 'AAA', 'AAAA', 'AAC', 'AACMP', 'AAE', 'AAF', 'AARAM', 'AARAW', 'AARAWEP', 'AAT', 'AAW', 'AB', 'ABA', 'ABB', 'ABBA', 'ABC', 'ABCCookingStudio', 'ABCCookingStudiom', 'ABCZ', 'ABS', 'ABeeC', 'AC', 'ACA', 'ACC', 'ACCENT', 'ACCESORIES', 'ACCESSCOLTD', 'ACCESSPORT', 'ACCS', 'ACDC', 'ACE', 'ACL', 'ACP', 'ACTOFVALOR', 'ACTUS', 'ACTUSIDEAinternationalBEAMSRECORDS', 'ACUBE', 'ACUBEAMD', 'ACUVUE', 'ACUVUER', 'ACbyAngelColor', 'AD', 'ADANGEROUSMETHOD', 'ADATA', 'ADC', 'ADDLayer', 'ADF', 'ADHOC', 'ADLatte', 'ADS', 'ADSL', 'ADSLCATV', 'ADUSB', 'ADUSBADUSB', 'ADVERTISING', 'ADayInTheLife', 'AE', 'AEP', 'AEPSON', 'AES', 'AF', 'AFAE', 'AFAX', 'AFC', 'AFCMOS', 'AFE', 'AFFS', 'AFLED', 'AFM', 'AFOMA', 'AFP', 'AFPBBNews', 'AFSSAPS', 'AFTERPARTY', 'AG', 'AGAC', 'AGAF', 'AGC', 'AGFAVista', 'AG

In [123]:
vect_data = vector.get_feature_names()

In [129]:
tfidf_vector = vector.transform(doc)

In [144]:
size = tfidf_vector.shape
size[0]

7376

###### cos類似度を用いて10個以上の文書対を比較する  => 類似度の高いものを探す

In [181]:
def cos_sim(doc1, doc2):
    #print("doc1:{} doc2:{}".format(doc1.shape, doc2.shape))
    #print(norm(doc1))
    return np.dot(doc1, doc2.T) / (norm(doc1) * norm(doc2))

In [188]:
def similarity(doc_ref, doc_all, num_of_doc):
    sim  = [[0 for _ in range(num_of_doc)] for _ in range(num_of_doc)]
    for i in range(num_of_doc):
            sim[i] = cos_sim(doc_ref, doc_all[i])
            
    return sim

文書[1010]に対して類似度の高い記事を探す。

In [189]:
cal_result = similarity(tfidf_vector[1010], tfidf_vector, size[0])

In [193]:
cal_result[1010] = 0

In [194]:
np.argmax(cal_result)

3678

###### 類似度の高かった文書

In [195]:
documents[3678]

'http://news.livedoor.com/article/detail/5695247/\n2011-08-10T11:00:00+0900\n勝負は最初の３分間。トークのツカミはOK？-人事担当者がこっそり教える採用ウラ話 vol.5\n転職者なら誰でも気になる採用する側の心理。しかし、実際に人事担当者から本音を聞きだすことはなかなかできないもの。そこでこのコーナーでは、多くの応募者を見てきた人事担当者に、意外な“採用ポイント”や心に残る採用エピソードなどをこっそり教えていただいた。\n\n\n勝負は最初の３分間。トークのツカミはOK？\n\n\n電機メーカー\n人事担当\n山田 和則氏（仮名）\nほとんどの採用者の方が皆さん口をそろえて言うと思いますが、面接において一番重要なのは、スバリ第一印象です。私の場合、スキルや実績は書類でだいたいわかりますから、面接ではその方の人柄の部分で合否を判断します。判断にかかる時間は、最初の３分間ってところじゃないでしょうか。\n\n「人は見かけだけじゃ分からないはず。もっと内面を見て欲しい」と思われる方もいるかもしれませんが、友達づきあいをするならともかく、一緒に仕事をするような間柄でいちいちそのひとの性格を理解してそれに合わせるってことはなかなかできないものです。特に営業など、社外の人間と多く接する職種の場合はより第一印象が重要だと思います。みなさんも自動車を購入するときなんかは、誠実そうな人から買いたいと思いますよね？ 実際、社会にでたら８割がたは外見で判断されると思っていいのではないでしょうか。\n\nもちろん、外見と言っても容姿が美しいとかおしゃれだとかそういうことではありません。その人の印象を一番左右するのは目でしょうかね。その人の目を見ればどのくらいうちの会社に入りたいと思ってくれているか、その本気さの度合いは大体分かりますから。やはり一緒に働くなら目がキラキラしている人と働きたいじゃないですか。あとはしぐさや話し方。「とりあえず受けに来てみました」っていう人はだいたいしぐさにもそれが表れてきます。逆に本当にうちの会社に入りたいと思ってくれている人は、自分のことを話すのでも工夫してこちらに伝えようとしてくれますし、面接官からも情報を聞き出す姿勢が感じられます。\n\n私は「あなたのいままでのキャリアヒスト

###### 入力した文書

In [196]:
documents[1010]

'http://news.livedoor.com/article/detail/5695257/\n2011-08-24T11:00:00+0900\n「あなたのモットーは？」に即答できますか？-人事担当者がこっそり教える採用ウラ話 vol.7\n転職者なら誰でも気になる採用する側の心理。しかし、実際に人事担当者から本音を聞きだすことはなかなかできないもの。そこでこのコーナーでは、多くの応募者を見てきた人事担当者に、意外な“採用ポイント”や心に残る採用エピソードなどをこっそり教えていただいた。\n\n\n「あなたのモットーは？」に即答できますか？\n\n\nアウトソーシング業界\n人事担当\n山下 雄三氏（仮名）\n私たち人事の仕事は、一人ひとりの人材の"目に見えない部分"を評価することだと思っています。資格や実務経験の有無といった目に見える部分は誰だって書類を見ればわかる。人事は採用のプロとして、それ以外のメンタル的な部分や潜在能力を見極め、適材適所を心掛けなければならないんです。ですから私は職務経歴書や履歴書はあまり重視しません。必要最低限の条件さえ満たしていれば面接で直接お会いして判断するようにしています。\n\nしかし面接でのジャッジは早いですよ。たいていの場合、数分でその人の"素養"はわかります。たとえば、差し出した名刺を着座したまま受け取ったり、会話中こちらの目を見なかったり、そうした些細なところにもその人の人柄というものは表れてくるんです。\n\nそういえば以前、ボロボロのジーンズに雪駄ばきという格好で面接を受けに来た方がいました。エンジニアだと私服で面接に来る方は多いんですが、これにはびっくりしましたよ。これ見よがしに個性を主張しようというならまだ分かるのですが（それも好ましくはありませんが）、どうやら本気でその格好がいいと思っているようで…。 でもやっぱり清潔感がないのはまずいですよねぇ（笑）。\n\n他にも「面接を受けるときは清潔感のあるスーツで」「面接官に進められるまで着席しない」といった新卒のときは当たり前だったマナーをすっかり忘れてしまっている人は結構います。でもこれは企業の責任でもあります。本当に魅力的な企業で、「どうしてもこの会社に入りたい」と思われるようならこうした態度はとられないでしょう？ マナーはその人の熱意の表れでもありますか

###### なぜそのような結果になったのか考察する

どちらの文章も転職者の面接について、最初の数分の印象が大事であること、面接者の意図を汲んで回答することが大事であるとが述べられておりかなり近い内容である。
どちらの文書にも、”転職”、”面接”、”印象”など同じような言葉が複数回使われており類似度が高いと判断されたと考えられる。

###### 【問題6】感情分析
テストデータに対する正解率が90%を超えるまで、調査=>実行=>改善を繰り返してください。
前処理になぜその処理をしたのかを書くとエンジニアリングとしても完璧です。

In [294]:
def text_cleaning2(text):
    tmp = remove_URL(text)
    tmp = remove_bracket(tmp)
    tmp = remove_bracket2(tmp)
    tmp = remove_emoji(tmp)
    cleand = normalize_number(tmp)
    return cleand

In [295]:
from sklearn.datasets import load_files

train_review = load_files('./aclImdb/train/', encoding='utf-8')
train_text, train_y = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
test_text, test_y = test_review.data, test_review.target

In [296]:
def text_vectorizer(text):
    vectorizer = TfidfVectorizer(analyzer="word")
    vectorizer.fit(text)
    return vectorizer

Cleaning : 感情予測に不要な（Pos/Negを表さない特殊文字、数字を削除する）

In [297]:
X = []
for i in range(len(train_text)):
    X.append(text_cleaning2(train_text[i]))

In [298]:
vectorizer = text_vectorizer(X)

In [299]:
train_x = vectorizer.transform(X)

In [300]:
train_x.shape

(25000, 71277)

In [301]:
X = []
for i in range(len(test_text)):
    X.append(text_cleaning2(test_text[i]))

In [302]:
test_x = vectorizer.transform(X)

In [303]:
test_x.shape

(25000, 71277)

In [305]:
from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import accuracy_score

In [306]:
#clf = svm.SVC(gamma='scale')
#clf = linear_model.LogisticRegression(random_state=0)

In [307]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(objective='binary',
                        num_leaves = 10,
                        learning_rate=0.1,
                        min_child_samples=1,
                        n_estimators=100)

In [312]:
get_mini_batch = GetMiniBatch(train_x, train_y, 100)
for epoch in range(50):
    for mini_X_train, mini_y_train in get_mini_batch:
        clf.fit(mini_X_train, mini_y_train)

In [313]:
y = clf.predict(train_x)

In [314]:
accuracy_score(y, train_y)

0.6532

結果、正解率は65.3%であった。

In [329]:
np_text = np.array(X)

In [358]:
correct = y[y == train_y]

In [359]:
worng = y[y != train_y]

In [334]:
correct_text = np_text[y == train_y]

In [331]:
worng_text = np_text[y != train_y]

###### 正解・不正解でのPosの比率（Posの方が予想し易いのか？それともNegの方か？）

In [357]:
pn_rate = np.sum(y) / len(y)
print("元データの Pos rate:",pn_rate)

元データの Pos rate: 0.58352


In [364]:
rate = np.sum(correct) / len(correct)
print("正解ケース Pos rate:",rate)

正解ケース Pos rate: 0.563931414574403


In [365]:
rate = np.sum(worng) / len(worng)
print("不正解ケース Pos rate:",rate)

不正解ケース Pos rate: 0.6204152249134948


データセット中で58%がポジティブデータであるの対し、正解データの中では56%、不正解だったデータの中では62%であった。  
よって、レビューのポジティブ・ネガティブによって感情分析結果の精度に大きな違いはない。（ネガティブなレビューの方が若干予測しやすいようだ）
以下、実際のレビューを見てみる。

###### レビューの中身を見てみる

正解だったレビュー

In [336]:
correct_text[0]

'I don\'t know how this movie has received so many positive comments. One can call it "artistic" and "beautifully filmed", but those things don\'t make up for the empty plot that was filled with sexual innuendos. I wish I had not wasted my time to watch this movie. Rather than being biographical, it was a poor excuse for promoting strange and lewd behavior. It was just another Hollywood attempt to convince us that that kind of life is normal and OK. From the very beginning I asked my self what was the point of this movie,and I continued watching, hoping that it would change and was quite disappointed that it continued in the same vein. I am so glad I did not spend the money to see this in a theater!'

In [337]:
correct_text[1]

"I caught this movie on the Horror Channel and was quite impressed by the film's Gothic atmosphere and tone. As a big fan of all things vampire related, I am always happy to see a new variation of the vampire mythos, in this case, a ghoul-like creature residing in a Lovecraftian other dimension. The director has done a brilliant job of conveying the dark mood of the subject, using the decadent art scene as a backdrop to what is essentially a tale of love spanning time and space- the pure love of friendship opposed to the lust for blood and life by the vampires in the story. The characters in the story are transported to another dimension by the means of a mind-altering substance, where a shape-shifting vampire creature appears to grant them their hearts desires, whilst draining them of their life essence. There are some analogies to drug addiction and loss of control, and how this affects a group of friends in an artistic circle. I enjoyed watching the 0 main male characters in the sto

In [338]:
correct_text[2]

'NBC had a chance to make a powerful religious epic along the lines of "The Ten Commandments" and "The Greatest Story Ever Told," and instead they chose to make some halfhearted cartoon that was more like "Waterworld" than anything else. I don\'t recall a Bible passage where Lot turns into a pirate and attacks the ark, nor do I remember one where Noah\'s son develops a serious friendship with an orange, nor do I remember Noah being some crazy old loon who suddenly acts like he\'s commanding a naval fleet and runs around shouting nautical terms like "hoist the mainstay!" This was possibly the worst marketing decision in history. Obviously the majority of people watching this were going to be Jewish and Christian parents with their kids, so why on earth make the movie so offensive to those people? If they were intentionally trying to offend, why not advertise it that way and at least reel in the right audience?? I hope they make a REAL Noah movie someday, one done seriously and thoughtfu

不正解だったレビュー

In [332]:
worng_text[0]

"Don't hate Heather Graham because she's beautiful, hate her because she's fun to watch in this movie. Like the hip clothing and funky surroundings, the actors in this flick work well together. Casey Affleck is hysterical and Heather Graham literally lights up the screen. The minor characters - Goran Visnjic {sigh} and Patricia Velazquez are as TALENTED as they are gorgeous. Congratulations Miramax & Director Lisa Krueger!"

In [333]:
worng_text[1]

"Looking for something shocking? Okay fine... the imagery is that. That's about it. This film attempts to make deep connections with the audience through various symbolism and just ends up being annoying. I am not quite sure if the director's purpose was to truly portray some sort of deep message to his audience, or if he just sought to shock the hell out of them with gore, sex and violence. I am thinking that it was probably the first...but in the failed attempt..it simply ended up to be a piece of artsy garbage with lots of blood, some obnoxious characters, and an over reliance on religious symbolism. If you're looking for some independent film to critique for its attempted use of metaphor...have at it. If you are looking for a gore flick that will make you queasy and uncomfortable... here you go... If you are looking for a film that will irritate you to no end because you realize that in the end, the message was stupid...the movie was stupid... and you will never get those minutes o

In [335]:
worng_text[2]

"I think this is one hell of a movie...........We can see Steven fighting around with his martial art stuff again and like in all Segal movies there's a message in it, without the message it would be one of many action/fighting movies but the message is what makes segal movies great and special."

不正解だったレビューには".....","!","{sigh}"など特殊文字が残ってしまっている。クリーニングが不十分だったために正解率が低かった可能性が考えられる。

###### 再クリーニング　＆　分析

In [372]:
def text_cleaning3(text):
    tmp = remove_URL(text)
    tmp = remove_bracket(tmp)
    tmp = remove_bracket2(tmp)
    tmp = remove_bracket3(tmp)
    tmp = remove_emoji(tmp)
    tmp = normal_char_pickup(tmp)
    tmp = ','.join(tmp)
    cleaned = tmp.replace(",", "")  
    return cleaned

In [373]:
X = []
for i in range(len(train_text)):
    X.append(text_cleaning3(train_text[i]))

In [374]:
vectorizer = text_vectorizer(X)

In [375]:
train_x = vectorizer.transform(X)

In [376]:
train_x.shape

(25000, 134332)

In [377]:
X = []
for i in range(len(test_text)):
    X.append(text_cleaning3(test_text[i]))

In [378]:
test_x = vectorizer.transform(X)

In [379]:
test_x.shape

(25000, 134332)

In [380]:
get_mini_batch = GetMiniBatch(train_x, train_y, 100)
for epoch in range(50):
    for mini_X_train, mini_y_train in get_mini_batch:
        clf.fit(mini_X_train, mini_y_train)

In [381]:
y = clf.predict(train_x)

In [382]:
accuracy_score(y, train_y)

0.62852

In [383]:
np_text2 = np.array(X)

In [384]:
correct2 = y[y == train_y]

In [385]:
worng2 = y[y != train_y]

In [386]:
correct_text2 = np_text2[y == train_y]

In [387]:
worng_text2 = np_text2[y != train_y]

In [390]:
worng_text2[2]

'I HATE THIS MOVIE I have never seen such utter complete trash in my life I live in France so it turned out that I was in the front line to watch this awful movie At first it seemed cool kind of like something about a cursed forest that chomps people Unfortunately it turned out it was something QUITE different Ancient war gods are lousy at kung fu All young women who study archeology at university in France are kung fu experts Professors are so resistant they can survive being sliced in two by a sawmasswhatchamacallit without any injuries'

###### 【問題7】分散表現
以下の中から一つ選んで分散表現を獲得し、
好きな単語群をt-SNE,PCAなどを用いて可視化してください。
コーパスは自由です。  
Word2Vec-CBoW(2〜3人)  
Word2Vec-skip-gram(2〜3人)  
fastText(2〜3人)  
また以下の4点についてもノートに書いてください。  
分布仮説とは何か？  
分散表現を得ることのメリットは何か？  
上で選んだモデルのメリット、デメリットは何か？  
なぜそのパラメータを選んだのか？  

###### Word2Vec-skip-gram

In [315]:
from gensim.models import word2vec
import logging

###### 分布仮説とは何か？

In [ ]:
同じ文脈で使われる単語は同じ意味を持つという仮説

###### 分散表現を得ることのメリットは何か？

単語をベクトル化することで演算が可能になる。演算によって類似度の算出、単語の予測などができる

###### 上で選んだモデルのメリット、デメリットは何か？

###### メリット  
アルゴリズムが簡単  

###### デメリット  
単語の意味を理解しているわけではない  
類似性と関連性を区別していない  
対義語に弱い  
学習していないデータに対応できない  

###### なぜそのパラメータを選んだのか？

###### 【問題8】自然言語処理の応用事例
NLPの情報共有
現在自然言語処理はどのような企業でどのように活用されているか？ 
1つ例をあげて3~5分で発表してください。 

https://www.ibm.com/developerworks/jp/cognitive/library/cc-watson-chatbot-conversation/index.html  
IBM Watoson Assistant